### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [1]:
%run D:\qtlab_replacement\init.py
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
from matplotlib import interactive
interactive(True)
%matplotlib qt5

Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


Чтобы не держать константы далеко в коде, мы записываем сюда параметры кубитов.

In [2]:
#Параметры кубитов.
Qubits = {1: {'Fr': 9.58578e9,'F01': 5.842543e9, 'I0': 0.,'Ispan': 0.,'P': -30,'Pex': -2,'Span': 200e6},
          2: {'Fr': 9.8075e9 ,'F01': 6.74974e9, 'P':-30},
          4: {'Fr': 9.587e9,'F01': 5.91557e9, 'I0': 0.,'Ispan': 0.,'P': -30,'Pex': -2,'Span': 200e6}}
qubit_id = 1

In [3]:
awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


# Спектроскопия

Инстантизируем драйверы устройств для векторного анализатора ('pna') и генератора сигналов ('lo1').

In [4]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'GEN1')

In [5]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Спектроскопия через смесители #### ЖМИ МИНЯ


Если подключена импульсная схема, а хочется сделать спектроскопию - то нет проблем! Единственное что нужно сделать, это, во-первых, открыть смесители (подав на них, скажем, 0.3В), и во-вторых, обратить внимание на мощность на выходе. Она будет ниже чем если делать спектроскопию нормально. Ну и шумов тоже получится больше - лучше всего не подавать какие-то сигналы с AWG, а задать оффсеты. Тогда вроде бы шумов меньше получается.

Что тут хорошо: можно переключаться между режимами не выходя из офиса (или из вообще не вставая на кровати). Этот режим очень популярен при измерениях в Черноголовке среди людей, которые не любят в эту самую Черноголовку ездить.
Тем не менее иногда полезно заходить в лабу и посмотреть что же на самом деле там происходит.

In [6]:
# AWG для того чтобы открыть смесители
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

# Задаём постояннmtые смещения на каналы 1 и 3
awg_tek.set_offset(0.3, channel=1)
awg_tek.set_offset(0.3, channel=3)

# Сами импульсы надо при этом выключить! Будет меньше шуметь (но то не точно).
awg_tek.stop()

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


## Резонаторная спектроскопия #### НЕЖМИ МИНЯ


In [ ]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-10) # 
pna.set_nop(1001)
pna.set_xlim(9.0e9, 10.0e9)
pna.set_bandwidth(100)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()

In [ ]:
meas = sweep.sweep(pna, ([0], lambda x: None, 'None'), filename='Single Tone spectroscopy #2')

## Резонатор от мощности (проверка нелинейности резонатора) #### НЕЖМИ МИНЯ


In [ ]:
pna.set_power(-10) # 
pna.set_nop(201) 
#pna.set_xlim(9.8e9, 9.81e9)
pna.set_xlim(9.4e9, 9.7e9)
pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

In [ ]:
resonator_powers = np.arange(16, -21, -1)
meas = sweep.sweep(pna, (resonator_powers, pna.set_power, 'Power, dBm'), filename='Single Tone spectroscopy power dependent #4')

## Фитуем добротность резонатора #### НЕЖМИ МИНЯ


In [ ]:
from resonator_tools import circuit
#plt.plot(freqs, scipy.signal.detrend(np.unwrap(S21[1])))
fitter =circuit.reflection_port(f_data=freqs, z_data_raw=S21[0]*np.exp(1j*S21[1]))
fitter.autofit()
fitter.plotall()

In [ ]:
fitter.fitresults['Qc_err']

In [ ]:
plt.plot(freqs, np.abs(np.gradient(S21[0]*np.exp(1j*S21[1]))))

In [ ]:
powers = np.linspace(-65, 5, 15)
sweep.sweep(pna, (powers, pna.set_power, 'readout power'), filename='Resonator power spectroscopy')

### 1D скан при фиксированной мощности

In [7]:
#### НЕЖМИ МИНЯ

qubit_id = 1
pna.set_nop(1)
#pna.set_xlim(9.58578e9, 9.58578e9)

True

In [9]:
#### НЕЖМИ МИНЯ

#Измерение
lo1.set_status(1)
ex_freqs = np.arange(5.7e9, 5.9e9, 1e6)

pna.set_xlim(Qubits[qubit_id]['Fr'], Qubits[qubit_id]['Fr'])
pna.set_nop(1)
pna.set_bandwidth(5)
#pna.set_average_mode('POINT')
#pna.set_average(1)
#pna.set_averages(1)
pna.set_power (-40)
lo1.set_power (10)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'), filename = "ex_{:.1f}dBm #2".format(lo1.get_power()))
except KeyboardInterrupt: pass 

A Jupyter Widget

Started at:  Apr 09 2018 16:04:34


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 1 m 21.04 s


In [ ]:
#### НЕЖМИ МИНЯ

#Измерение
lo1.set_status(1)
ex_freqs = np.arange(5.9e9, 6.0e9, 2e6)

pna.set_xlim(9.45e9, 9.75e9)
pna.set_nop(1001)
pna.set_bandwidth(501)
pna.set_average_mode('POINT')
pna.set_average(1)
pna.set_averages(1)
pna.set_power (-15)
lo1.set_power (13)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm #4".format(lo1.get_power()))
except KeyboardInterrupt: pass 

##### 2D скан (мощность второго тона, частота второго тона)

In [ ]:
#### НЕЖМИ МИНЯ
Qubit_id = 2
ex_freqs = np.arange(6.25e9, 6.9e9, 2e6)
ex_powers = np.arange(-20, 15, 5)

pna.set_xlim(Qubits[Qubit_id]['Fr'], Qubits[Qubit_id]['Fr'])
pna.set_nop(1)
pna.set_power(Qubits[Qubit_id]['P'])
pna.set_bandwidth(5)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           header = header)
lo1.set_status(0)

# Импульсы #### ЖМИ МИНЯ ПОЛНОСТЬЮ

### Создаём девайсы

Инстантизация драйверов устройств.
sa - анализатор спектра,
awg_tek - геннератор сигналов произвольной формы,
adc - аналогово-цифровой преобразователь.

In [6]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'PXA')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
#awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')


### Загружаем импульсные скрипты всякие

In [7]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

Генераторы опорных волн для смесителей.
На вход надо подавать около 13 дБ (точка децибельной компрессии).
В качестве источника можно использовать agilent или labbrick. У labbrick фазовые шумы, лучше использвать agilent.
Векторный анализатор можно включить в режим постоянной волны, тогда его тоже можно использовать.

In [8]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)
awg_tek=awg

### AWGшки

In [9]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 125e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 0.7 В на считывание, 0.4 В на возбуждение.
for channel in range(3,5):
    awg_tek.set_amplitude(0.5, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
for channel in range(1,3):
    awg_tek.set_amplitude(0.8, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 2, 1, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[qubit_id]['F01'])
iq_ro.set_frequency(Qubits[qubit_id]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды

ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [10]:
# настройки оцифровщика
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())
# Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
adc_reducer.extra_opts['scatter'] = True

### Измерение двухтонового спектра импульсами

In [45]:
#### НЕЖМИ МИНЯ

#iq_ro.awg_I.__set_dc(1.0)
frequencies = np.arange(5.5e9, 5.9e9, 2e6)
iq_ex.set_if(0.0)
iq_ex._set_dc(0.1)

measurement = sweep.sweep(adc_reducer, (frequencies, iq_ex.set_frequency, 'Second tone frequency'), 
                          filename='Two-tone mixer passthrough')

A Jupyter Widget

Started at:  Apr 09 2018 14:35:33


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

root: ERROR    Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\qtlab_replacement\scripts\sweep.py", line 184, in sweep
    plotting.update_plot_measurement(mk_measurement(0), plot_axes)
  File "D:\qtlab_replacement\scripts\plotting.py", line 138, in update_plot_measurement
    plots = update_plot_measurement_sa(meas, ax)
  File "D:\qtlab_replacement\scripts\plotting.py", line 247, in update_plot_measurement_sa
    plt.clim(np.nanmin(data), np.nanmax(data))
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py", line 2226, in clim
    raise RuntimeError('You must first define an image, e.g., with imshow')
RuntimeError: You must first define an image, e.g., with imshow

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-45-122e72e2a5d8>", line 9, in <m

TypeError: must be str, not list

### Калибровка миксеров

In [11]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

d:\qtlab\ReiData\data/calibrations//IQ-if1.2e+08-rf5.8e+09-sb--1.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if7.5e+07-rf9.6e+09-sb--1.pkl


{'I': (0.31353201731277081+0.54257746444126442j),
 'Q': (0.57193952776845824+0.51228166137994147j),
 'dc': (0.043093863426735338-0.045260680954120322j),
 'num_sidebands': 7,
 'score': -18.798982445552799}

### Длительность и амплитуда считывающих импульсов

In [12]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(8./7.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*7/8
    
    print (nop, type(nop))
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [26]:
adc.stop()

In [13]:
 
# ro_amplitude = 0.5 Qubit #2 preliminary
# ro_amplitude = 0.3 Qubit #3 preliminary
ro_dac_amplitude = 0.25
ro_dac_length =0.1e-6 # вообще считывающие импульсы такой длины? тогда что такое adc 
# Ответ: ro_dac_length - это длительность генерируемого прямоугольного импульса
# ro_adc_length - это минимальное время, в течении которого мы считываем этот импульс, когда он прошёл через криостат.
# В криостате импульс деформируется и становится длинее. Сколько времени хотите его интегрировать, столько и ставьте.
# Наверное, имеет смысл сюда поставить на 1-2 времени затухания фотона в резонатор больше, чем ro_dac_length
#ro_adc_length = ro_dac_length*3
#ro_adc_length=0.8e-6 use it for bench in the end
ro_adc_length=0.6e-6  
#adc.set_nums(30000)
#adc.set_software_nums_multi(1)
#set_adc_nop(ro_adc_length)
adc.set_nums(1000)
set_adc_nop(ro_adc_length)
adc.set_nums(30000)


512 <class 'int'>


True

In [19]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.figure()
plt.plot(np.mean(np.imag(adc.measure()['Voltage']), axis=0))

KeyboardInterrupt: 

In [ ]:
plt.pcolormesh(np.real(adc.measure()['Voltage']));
plt.colorbar()

In [ ]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [28]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 21), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

A Jupyter Widget

Started at:  Apr 09 2018 16:16:16


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 1 m 31.78 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [35]:
adc.stop()

### Осцилляции Раби

In [36]:
ex_amplitude = 1.0#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0e-6, 1.0e-6,101)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters_rabi['phase'], 
                                                                             fitted_parameters_rabi['freq'], 
                                                                             fitted_parameters_rabi['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

A Jupyter Widget

Started at:  Apr 09 2018 16:35:22


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 7 m 4.98 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

In [ ]:
Rabi_fr=fitted_parameters_rabi['freq']
T_pi=np.pi/Rabi_fr
T_pi_half=T_pi/2
T_pi,T_pi_half

In [ ]:
measurement

In [ ]:
measurement_fitted

In [ ]:
Rabi_fr

In [ ]:
det=np.sqrt(4*Rabi_fr**2-Qubits[4]['F01']**2)
det

### Осцилляции Рамзея

In [ ]:
ex_amplitude = 1.0
# pi2_pulse = 7e-9 #Q2
pi2_pulse = 64e-9 #Q4
#pi2_pulse = T_pi_half #Q4
pause_length = 16e-9
delays = np.linspace(0e-9, 1000e-9, 51)#последовательность задержек между двумя пи/2 импульсами
target_freq_offset = 3e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)),#зачем фаза? 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay', 's'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Apr 09 2018 18:01:40


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


In [45]:
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.2
pi_pulse = 20e-9
pause_length = 16e-9

In [56]:
adc.stop()

In [57]:
pi_pulse = 128e-9
delays = np.linspace(0e-6,8e-6, 51)#серия задержек после возбуждения перед считыванием состояния
readout_begin = np.max(delays)+pi_pulse#зачем он тут прибавляет, а потом в коде вычитает? 
# раньше смысл был в том, чтобы менять длительность возбуждающей последовательности, не сдвигая считывание. То есть
# дополнительные импульсы приделываются до считывания (котороене двигается), а не считывание приделывается к концу.
# Сейчас это ещё раз реализовано в коде, которые делает последовательности импульсов (pg.set_seq). Если вдруг у нас будет
# много считываний, то там тоже надо будет мутить какую-нибудь синхронизацию.
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi_pulse),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Apr 09 2018 17:55:26


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 3 m 35.96 s
[7.9202813932156325e-06, -0.06763574113038846, -0.03240679379727257, 0.05780554537855299, -0.037322928960993804, -0.06063712100194457, -0.03310670786357711, 0.058125560513415594, -0.03716962968677179]


In [53]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


# Double readout

In [ ]:
adc.set_nums(10000)

sweep_number=5
result=[]
for swp in range(sweep_number):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    measurement=adc.measure()
    #result.append(measurement)
    with open('data.pickle{0}'.format(swp), 'wb') as f:
        pickle.dump(measurement, f)
   # measurement.clear()
    print('{0} done'.format(swp))
    



In [ ]:
#ex_amplitude = 0.2
#pi_pulse = 20e-9
#pause_length = 16e-9
#pi_pulse=20e-9
adc.set_nums(10000)


sweep_number=5
result=[]
for swp in range(sweep_number):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha),
                pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha),
                pg.p(None,pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    measurement=adc.measure()
   # result.append(measurement)
    with open('data1gaus.pickle{0}'.format(swp), 'wb') as f:
        pickle.dump(measurement, f)
    measurement.clear()
    print('{0} done'.format(swp))
    

In [ ]:
imp.reload(sweep)
imp.reload(save_pkl)
sweep_number=100
adc.set_nums(500)
measurment_data_pre=[]
for swp in range(sweep_number):
    delays = np.linspace(0e-6, 10e-6, 51)
    def set_delay(delay):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
        sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude),
                    pg.p(None, delay),
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)
                   ]
        pg.set_seq(sequence)
        awg_tek.run()
    measurement_pre = sweep.sweep(adc, (delays, set_delay, 'delay'), filename='Decay{0}'.format(swp),time_war_label=False)
    measurement_pre.clear()
   
    
 


In [ ]:
for file_name in range (swp):
    if (file_name==0):
        with open(r'D:\qtlab\ReiData\data\2017-11-03\Voltage Decay{0}.pkl'.format(file_name),'rb') as f:
            data=pkl.load(f,encoding='latin1')
        Voltage_avg=[[np.mean(data[1]['Voltage'][2][i][j]) for i in range(len(data[1]['Voltage'][2][0][1]))]\
                     for j in range(len(data[1]['Voltage'][2][0]))]
        print (Voltage_avg_amp[0])
        print (file_name)
    else:
        with open(r'D:\qtlab\ReiData\data\2017-11-03\Voltage Decay{0}.pkl'.format(file_name),'rb') as f:
            data_r=pkl.load(f,encoding='latin1')
        Voltage_avg=[(np.real(data_r[1]['Voltage'][2][vnum])+np.real(Voltage_avg[vnum]))/2+\
                      (np.imag(data_r[1]['Voltage'][2][vnum])+np.imag(Voltage_avg[vnum]))/2 for vnum \
                      in range(len(data_r[1]['Voltage'][2]))]
        print (data_r[1]['Voltage'][2])
        print (Voltage_avg[0])
        print (file_name)
                      

In [ ]:
sweep_num=80
adc.set_nums(500)
delays = np.linspace(0e-6, 10e-6, 51)
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, delay)
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude),
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement_pre = sweep.sweep(adc, (delays, set_delay, 'delay'), filename='Decay')
voltage_amp_pre_ONES=[np.sqrt(np.real(measurement_pre['Voltage'][2][1][i])**2+np.imag(measurement_pre['Voltage'][2][1][i])**2) for i in \
             range(len(measurement_pre['Voltage'][2][1]))]
voltage_phase_pre_ONES=[np.arctan(np.real(measurement_pre['Voltage'][2][1][i])/np.imag(measurement_pre['Voltage'][2][1][i])) for i in \
             range(len(measurement_pre['Voltage'][2][1]))] 
plt.plot(voltage_amp_pre_ONES)
plt.show()

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
non_avg_ones=[[voltage_amp_pre_ONES[i][j] for i in range(len(voltage_amp_pre_ONES))]\
              for j in range (len(voltage_amp_pre_ONES[0]))]
avg_ones=[np.mean(ones) for ones in non_avg_ones ]
non_avg_zeros=[[voltage_amp_pre_ZEROS[i][j] for i in range(len(voltage_amp_pre_ZEROS))]\
              for j in range (len(voltage_amp_pre_ZEROS[0]))]
avg_zeros=[np.mean(zeros) for zeros in non_avg_zeros ]
avg_ones_n=[np.mean(ones)-np.mean(avg_ones) for ones in non_avg_ones ]
avg_zeros_n=[np.mean(zeros)-np.mean(avg_zeros) for zeros in non_avg_zeros ]
plt.plot(avg_ones_n)
plt.plot(avg_zeros_n)
blue_patch = mpatches.Patch(color='blue', label='|1>')
red_patch = mpatches.Patch(color='orange', label='|0>')
plt.legend(handles=[blue_patch,red_patch])
plt.show()

In [ ]:
delays = np.linspace(0e-6, 1.5e-6, 51)
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude),
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude) ]
    pg.set_seq(sequence)
    awg_tek.run()
measurement_post = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

In [ ]:
voltage_amp_post=[np.sqrt(np.real(measurement_post['Voltage'][2][1][i])**2+np.imag(measurement_post['Voltage'][2][1][i])**2) for i in \
             range(len(measurement_post['Voltage'][2][1]))]
voltage_phase_post=[np.arctan(np.real(measurement_post['Voltage'][2][1][i])/np.imag(measurement_post['Voltage'][2][1][i])) for i in \
             range(len(measurement_post['Voltage'][2][1]))]

In [ ]:
measurement

In [ ]:
measurement['S21-'][1][0]
S21_pre=measurement['S21-'][1][0][::2]
S21_post=measurement['S21-'][1][0][1::2]
len(measurement['S21-'][1][0])

# Optimizing readout pulse settings

In [24]:
#+ 
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
# Кубит #4, импульсы 8 нс
#pause_length = 2e-9
#length = 10e-9 # Длительность импульса 
#sigma = 2.5e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.893
#amp_y=0.880
#anharmonicity = -200e6
#alpha_dimensionless = 0.724
# Кубит #4, импульсы 16 нс
#pause_length = 0e-9
#length = 16e-9 # Длительность импульса 
#sigma = 3e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.322
#anharmonicity = -200e6
#alpha_dimensionless = 0.909#553
#alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
#xy_acorr = 0.003
#amp_y_raw=0.322#
amp=0.3245
amp2=0.655
length=16e-9
sigma=3e-9
anharmonicity=-227e6
pause_length=0
alpha_dimensionless =1.035
# alpha_dimensionless =1.1
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
xy_acorr =0.0
amp_y_raw=0.3245
amp_y=amp_y_raw*np.exp(1j*xy_acorr)

In [ ]:
#parametres
ampx_pi_half
ampx_pi
ampy_pi_half
ampy_pi
ength=16e-9
sigma=3e-9
anharmonicity=-227e6
alpha_dimensionless =1.035
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)

In [25]:
# To calibrate we need a pi-pulse. Two choices are available, either we use a rectangular pulse or a gaussian pulse.
# The pulse is used to excite the qubit with high probability.
#ex_calib_seq = [pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude)] # rectangular pulse
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma), pg.p(None, pause_length)]*2 # gaussian pulse
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc.set_software_nums_multi(1)
adc_sz.repeat_samples = 1

adc.set_nums(50000)

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_dac_amplitude, ro_dac_length):
        self.ro_dac_amplitude = ro_dac_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0, 0.05e-6, 6)
ro_dac_amplitudes = np.linspace(0.0,0.5, 11)
#ro_dac_length = 300e-9

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_dac_amplitude, ro_dac_length)
#solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
#                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)

#ro_dac_length = solution[0]
#ro_dac_amplitude = solution[1]

In [26]:
ro_amplitude_calib = sweep.sweep(adc_sz, #(ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'))
ro_dac_amplitude = ro_amplitude_calib['Calibrated fidelity'][1][0][np.argmax(ro_amplitude_calib['Calibrated fidelity'][2])]

adc.set_software_nums_multi(1)

Started at:  Nov 10 2017 17:47:17


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:642: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
D:\qtlab_replacement\scripts\tomography.py:123: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all



Elapsed time: 0 h 3 m 36.2 s


True

In [27]:
#setting optimal settings 
print ('Calibrated optimal readout amplitude: ', ro_dac_amplitude)
ro_pulse_generator_inst.set_ro_amplitude(ro_dac_amplitude)
adc_sz.measure()
plt.plot(adc_sz.calib_proba_points, adc_sz.calib_hists.T)
adc_calibrated_reducer = data_reduce.data_reduce(adc)
adc_calibrated_reducer.filters['sz'] = adc_sz.filter

Calibrated optimal readout amplitude:  0.4


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:642: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


In [ ]:
adc.stop()
pg.p

### Rabi amplitude oscillations

In [ ]:
#ex_amps = np.linspace(0.3225,0.3325,21)
#num_pulses = 50
ex_amps = np.linspace(0.1,0.7,41)
num_pulses = 4
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

## Rabi amplitude oscillations pi + pi/2 pulses

In [35]:
from numpy import pi
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
#measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
#measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
#def find_ampl(impulse):
    #i=0
    #derivative1, derivative2 =0,0
    #count=0
    #while (0.5*count<impulse/(0.5*pi)):
        #i=i+1;
        #derivative1=derivative2;
        #derivative2=(3*measurement['Mean Voltage (AC)'][i]-4*measurement[i-1]+measurement[i-2])/(2*(0.7/21)
        #if ((derivative1*derivative2)<0) :
            #count=count+1
    #return measurement[i]

#def Rabi_ampl(impulse):

# В таком сиде как оно написано здесь эта процедура не имеет никакого смысла =(
# Если утверждение amp*2=amp_pi верно, то это не нужно. Если нет, то таким обрзом вы это не увидите.

ex_amps = np.linspace(0,0.7,41)
num_pulses=4
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
print(fitted_parameters_rabi)
      # return find_ampl(impulse)
#ampl_half_pi=Rabi_ampl(pi/2)   
amp=(3*pi-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp_pi=(pi*5-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp, amp_pi

Started at:  Nov 10 2017 18:17:17


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:642: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 4 m 36.72 s
{'phase': 3.2075903075746095, 'freq': 3.0276699017788729, 'decay': 12.667307609565038, 'amplitudes': array([  1.03572704e-03,   1.20872809e-05,  -9.75184365e-04,
         1.22408749e-02])}


(0.32681771105710067, 0.65710470813444433)

### Rabi Y-amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.1,0.7, 101)
num_pulses = 5
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi Y-pulse amplitude'), filename='Rabi Y-amplitude')

## Amplification of phase errors

In [ ]:
num_pulses = np.arange(0, 201, 21)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1

def set_ape_length(num_pulses):
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]*num_pulses+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (num_pulses, set_ape_length, 'APE identity pulse num'), filename='APE Y'+y_sign)

# Scanning over alpha and looking at phase error

In [ ]:
num_pulses = 80
y_sign = '+'
y_sign_mul = 1 if y_sign == '+' else -1
#anharmonicity = -223e6
#alphas = np.linspace(-1.0, 1.0, 21)/(200e6*2*np.pi)
alphas = np.linspace(0.75, 1.2, 21)

def set_alpha(alpha_dimensionless):
    alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]*num_pulses+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (alphas, set_alpha, 'HD DRAG alpha'), filename='APE Y'+y_sign+' dependence on HD DRAG alpha')

### X-Y axes angle error

In [ ]:
num_pulses = np.arange(0, 21, 1)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1

def set_xyea_length(num_pulses):
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (num_pulses, set_xyea_length, 'X-Y axes angle error pulse num'), filename='XY angle error witness'+y_sign)

### Optimizing X-Y axes angle 

In [ ]:
#angle_corrs = np.linspace(-np.pi/40, np.pi/40, 21)
angle_corrs = np.linspace(-0.03, 0.03, 21)
num_pulses = 20

for signs in ['++++', '+++-', '++-+', '++--', '+-++', '+-+-', '+--+', '+---', 
              '-+++', '-++-', '-+-+', '-+--', '--++', '--+-', '---+', '----']:
    sign_muls = [1 if s == '-' else -1 for s in signs]

    def set_xyea_angle_corr(angle_corr):
        sequence = [pg.p('iq_ex', length, pg.gauss_hd, sign_muls[0]*amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[3]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

        pg.set_seq(sequence)
        awg_tek.run()
    measurement = sweep.sweep(adc_reducer, (angle_corrs, set_xyea_angle_corr, 'X-Y axes angle detuning (radians)'), filename='XY angle error optimization'+signs)

## Characterizing readout of |0>, |1>, |+>, |->, |i+>, |i-> states

In [ ]:
state_prep_seqs = {'|0>': [],
                   '|+>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|1>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),\
                           pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|->': [pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|i+>': [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)],
                   '|i->': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y,sigma, alpha), pg.p(None, pause_length)]}

plt.figure('State readout histograms gaussian')
sz_meas = {}
for state_name, state_prep_seq in state_prep_seqs.items():
    pg.set_seq(state_prep_seq+adc_sz.ro_seq)
    sz_meas[state_name] = np.real(adc_calibrated_reducer.measure()['sz'])
    hist, bin_edges = np.histogram(sz_meas[state_name], bins=adc_sz.calib_proba_points)
    proba_points = (bin_edges[1:]+bin_edges[:-1])/2
    plt.plot(proba_points, hist,label=state_name)
    print ('{0} state sz mean: {1}, std: {2}'.format(state_name, np.mean(sz_meas[state_name]), np.std(sz_meas[state_name])))
    
plt.legend()

In [ ]:
adc.stop()

### Tomography

In [41]:
#+ 
amp=0.327
amp2=0.657
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [43]:
#+Gh
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
reconstruction_basis={'z':{'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [40]:
## tomography of |0> state
#adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:642: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


KeyboardInterrupt: 

# Clifford group

In [44]:
#+ в оптимальных параметрах амплитуду на 2 умножили и получили пи импульсы))))))000)))
import clifford
imp.reload(clifford)

pi2 = {'X/2': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
       'Y/2': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
       '-X/2':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
       '-Υ/2':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
       'I':   {'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])},
       'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp2, 0, sigma, alpha), pg.p(None, pause_length)],
               'unitary': np.asarray([[0, 1],  [1, 0]])},
       'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp2, sigma, alpha), pg.p(None, pause_length)],
               'unitary': 1j*np.asarray([[0, -1],  [1, 0]])}}
clifford_group = clifford.generate_group(pi2)#тут меняем наш словарь
print(clifford_group.keys(), len(clifford_group))

dict_keys(['X/2', 'Y/2', '-X/2', '-Υ/2', 'I', 'X', 'Y', 'Y/2 X/2', '-Υ/2 X/2', 'Y X/2', 'X/2 Y/2', 'X/2 Y/2 X/2', '-X/2 Y/2', '-X/2 Y/2 X/2', 'X Y/2', '-Υ/2 X/2 Y/2', 'Y/2 -X/2', 'Y/2 -X/2 Y/2', '-Υ/2 -X/2', 'Y -X/2', 'X/2 -Υ/2', '-X/2 -Υ/2', 'X -Υ/2', 'Y X']) 24


In [ ]:
import qutip as qt
qt.sigmay()

# Randomized benchmarking

In [45]:
#+
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)
imp.reload(sweep)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

#unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
#             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
#             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
#             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = clifford_group
#bench.sequence_length = 10
bench.random_sequence_num = 20
#bench.prepare_random_interleaving_sequences()
#bench.get_points()
#bench.generate_interleaver_sequence_from_names()

In [50]:
adc.stop()

In [54]:
#+
#ro_adc_length=0.2e-6 #ВНИМАНИЕ! я это удалю (закомментирую) тут, т.к. оно всё равно здесь ничего не делает, только мозги выносит
seq_lengths = np.arange(0,101, 20)
#seq_lengths = np.arange(0, 100, 10)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
clifford_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

Started at:  Nov 10 2017 18:55:23


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:642: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 13 m 18.76 s


In [69]:
#+
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 
         1-clifford_bench['distance'][2], 
         marker='o', 
         color='black', 
         linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1))
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))

plt.plot(clifford_bench['distance'][1][0],
         clifford_expfit[0],
         label='Clifford set')
plt.grid()
plt.xlabel('Number of gates')
plt.ylabel('Fidelity')
plt.legend()
print('Clifford fidelity: {0:6.3f}'.format(np.exp(-1/clifford_fitresults[0])))

Clifford fidelity:  0.989


In [82]:
clifford_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-07-16'
name='distance None'
clifford_bench = {}
clifford_bench = save_pkl.load_pkl(name, clifford_location)[1]
#clifford_bench['distance']
clifford_bench['Pulse sequence']

D:\qtlab\ReiData\data\2017-11-02\21-07-16/distance None.pkl


(('Gate number', 'Random sequence id'),
 (array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50]),
  array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])),
 array([[['I'], ['I'], ['I'], ['I'], ['I'], ['I'], ['I'], ['I'], ['I'],
         ['I']],
        [ ['-X/2 -Υ/2', 'Identity (benchmarking)', '-Υ/2', 'Identity (benchmarking)', 'X/2 Y/2', 'Identity (benchmarking)', 'Y/2 X/2 X/2', 'Identity (benchmarking)', 'Y/2', 'Identity (benchmarking)', '-Υ/2'],
         ['Y/2 Y/2', 'Identity (benchmarking)', '-Υ/2', 'Identity (benchmarking)', '-X/2 Y/2 X/2', 'Identity (benchmarking)', '-Υ/2 X/2 Y/2', 'Identity (benchmarking)', 'X/2 Y/2', 'Identity (benchmarking)', 'X/2 X/2'],
         ['-X/2 Y/2 X/2', 'Identity (benchmarking)', 'X/2 -Υ/2', 'Identity (benchmarking)', '-Υ/2', 'Identity (benchmarking)', 'X/2 Y/2 Y/2', 'Identity (benchmarking)', 'X/2 Y/2 X/2', 'Identity (benchmarking)', 'X/2 X/2'],
         ['Y/2 Y/2', 'Identity (benchmarking)', 'Y/2 X/2 X/2', 'Identity (benchmarking)', 'X/2', 'Identity (benchmarking)',

In [89]:
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])

[['-X/2 -Υ/2', '-Υ/2', 'X/2 Y/2', 'Y/2 X/2 X/2', 'Y/2', '-Υ/2'],
 ['Y/2 Y/2', '-Υ/2', '-X/2 Y/2 X/2', '-Υ/2 X/2 Y/2', 'X/2 Y/2', 'X/2 X/2'],
 ['-X/2 Y/2 X/2', 'X/2 -Υ/2', '-Υ/2', 'X/2 Y/2 Y/2', 'X/2 Y/2 X/2', 'X/2 X/2'],
 ['Y/2 Y/2', 'Y/2 X/2 X/2', 'X/2', 'X/2', 'Y/2 X/2', '-X/2'],
 ['Y/2 Y/2', 'X/2 -Υ/2', 'Y/2 X/2 X/2', 'Y/2 -X/2 Y/2', '-Υ/2', 'I'],
 ['-X/2 -Υ/2', 'X/2 Y/2 Y/2', 'X/2 Y/2', '-Υ/2 -X/2', '-Υ/2', '-Υ/2'],
 ['X/2', '-X/2 Y/2 X/2', 'X/2 Y/2', 'X/2 -Υ/2', 'Y/2 X/2', 'I'],
 ['Y/2 -X/2', 'Y/2 X/2 X/2', 'X/2 -Υ/2', 'X/2 Y/2 X/2', 'Y/2 -X/2 Y/2', 'X/2'],
 ['X/2 Y/2 Y/2',
  'Y/2 -X/2 Y/2',
  'X/2 X/2',
  'Y/2 Y/2',
  'Y/2 Y/2 X/2 X/2',
  '-Υ/2'],
 ['Y/2 -X/2 Y/2',
  'Y/2 Y/2 X/2',
  '-Υ/2 X/2 Y/2',
  '-Υ/2 X/2 X/2',
  '-Υ/2 -X/2',
  'Y/2']]

In [90]:
clifford_bench['distance'][2][1,:]

array([ 0.31   ,  0.32096,  0.31178,  0.31158,  0.30912,  0.31068,
        0.31074,  0.3155 ,  0.31622,  0.32042])

In [73]:
not_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\22-14-16'
name='distance None'
not_bench = {}
not_bench = save_pkl.load_pkl(name, not_location)[1]
#clifford_bench['distance']
pass

D:\qtlab\ReiData\data\2017-11-02\22-14-16/distance None.pkl


In [75]:
hadamard_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-40-30'
name='distance None'
hadamard_bench = {}
hadamard_bench = save_pkl.load_pkl(name, hadamard_location)[1]
#clifford_bench['distance']
pass

D:\qtlab\ReiData\data\2017-11-02\21-40-30/distance None.pkl


## interleaved benchmarking of H gate and NOT

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 10
bench.target_gate = clifford_group['Y/2 X/2 X/2']['pulses'] # NOT
bench.target_gate_name = 'Hadamard'
bench.target_gate_unitary = clifford_group['Y/2 X/2 X/2']['unitary']


In [ ]:
seq_lengths = np.arange(0, 51, 5)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
hadamard_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

In [78]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (average)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='Hadamard interleaved (average)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
Hadamard error rate: {2:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

Clifford fidelity:  0.989
Hadamard+Clifford fidelity:  0.982
Hadamard error rate:  0.007


In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 10
bench.target_gate = clifford_group['X/2 X/2']['pulses'] # NOT
bench.target_gate_name = 'NOT'
bench.target_gate_unitary = clifford_group['X/2 X/2']['unitary']


In [ ]:
seq_lengths = np.arange(0, 51, 5)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
not_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

In [81]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (mean)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='H interleaved (mean)')
plt.plot(not_bench['distance'][1][0], 1-not_bench['distance'][2], marker='o', color='red', linestyle='')
#plt.plot(not_bench['distance'][1][0], np.mean(1-not_bench['distance'][2], axis=1), label='NOT interleaved (mean)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
not_fitresults, not_expfit = fitting.exp_fit1d(not_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-not_bench['distance'][2], axis=1), 
                                                                   (1, len(not_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
plt.plot(not_bench['distance'][1][0],not_expfit[0], label='NOT (fit)')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
NOT+Clifford fidelity: {3:6.3f}
Hadamard error rate: {2:6.3f}
NOT error rate: {4:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/not_fitresults[0]),
                                       1-np.exp(-1/not_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

Clifford fidelity:  0.989
Hadamard+Clifford fidelity:  0.982
NOT+Clifford fidelity:  0.984
Hadamard error rate:  0.007
NOT error rate:  0.005


In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))